In [ ]:
import os
import re
import csv

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


import tensorflow as tf
tf.random.set_seed(99)

import grpc
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [ ]:
CLASSES = ['aim', 'email', 'facebook', 'ftps', 'gmail', 
           'hangout', 
           'icqchat',
           'netflix', 'scp', 'sftp',
           'skype', 'spotify', 'torrent', 'vimeo', 'voipbuster',
           'youtube']

In [ ]:
def do_predict(data):
    channel = grpc.insecure_channel('localhost:8500')
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
    request = predict_pb2.PredictRequest()
    request.model_spec.name = '1'
    request.model_spec.signature_name = 'serving_default'
    request.inputs['input_1'].CopyFrom(tf.make_tensor_proto(data, shape=[1, 375, 4, 1]))
    request_future = stub.Predict.future(request, 30)
#     return stub.Predict(request)
    result = request_future.result()
    return CLASSES[np.argmax(result.outputs['dense'].float_val)]

In [ ]:
def get_files(root_dir, ext=('.jpg', '.png')):
    queue = [root_dir]
    while len(queue) != 0:
        nest_dir = queue.pop()
        label = None
        with os.scandir(nest_dir) as it:
            for entry in it:
                if not entry.name.startswith('.') and entry.is_file():
                    if entry.name.endswith(ext):
                        label = os.path.basename(os.path.dirname(entry.path)) # dirname is label
                        yield label, entry.path
                elif not entry.name.startswith('.') and entry.is_dir():
                    queue.append(entry.path)

In [ ]:
def get_data(path):
    with open(path, 'rb') as f:
        data = plt.imread(f)
    return data

In [ ]:
f = open('predict_result.csv', 'w')
writer = csv.DictWriter(f, fieldnames=['Path', 'Label', 'Predict'],
                        quoting=csv.QUOTE_MINIMAL,
                        lineterminator=os.linesep)
writer.writeheader()

cnt = 0
for label, path in get_files('dataset/img_data'):
    index = int(re.split('[.-]', path)[-2])
    if index > 3:
        continue
    label = label.lower()
    data = get_data(path)
    result = do_predict(data)
    result = result.lower()
    writer.writerow({'Path': os.path.basename(path),
                     'Label': label,
                     'Predict': result})
f.close()

In [ ]:
f = open('predict_result.csv', 'r')
reader = csv.DictReader(f)

cnt = [0, 0] # T F
do = dict()
for row in reader:
    s = row['Label'] == row['Predict']
    if s:
        cnt[0] += 1
    else:
        cnt[1] += 1
    name = row['Path'].split('.')[0]
    rec = do.get(name, [0, 0])
    if s:
        rec[0] += 1
    else:
        rec[1] += 1
    do[name] = rec

f.close()

In [ ]:
cnt2 = [0, 0]
for v in do.values():
    if v[0] > v[1]:
        cnt2[0] += 1
    else:
        cnt2[1] += 1

In [ ]:
print(cnt, cnt[0]/sum(cnt))
print(cnt2, cnt2[0]/sum(cnt2))

In [ ]:
sum(cnt2)